<a href="https://colab.research.google.com/github/adityasahusomu/Youtube_Comment_Analyzer/blob/main/experiment_4_handling_imbalanced_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 91.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 79.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.2/86.2

In [2]:
import os, getpass

os.environ["AWS_ACCESS_KEY_ID"] = getpass.getpass("Enter AWS Access Key ID: ")
os.environ["AWS_SECRET_ACCESS_KEY"] = getpass.getpass("Enter AWS Secret Access Key: ")
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

Enter AWS Access Key ID: ··········
Enter AWS Secret Access Key: ··········


In [3]:
import boto3
s3 = boto3.client('s3')
print(s3.list_buckets())

{'ResponseMetadata': {'RequestId': 'RFV6RBG6NDPVD7NY', 'HostId': 'FuxeceOWRd+ktBRpnGJ+RlUQ4GzLlVnsyD9RwQ4klp8ZN8gkuXoFbp8Tf+h8IfpanEA+K+EfEL0=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'FuxeceOWRd+ktBRpnGJ+RlUQ4GzLlVnsyD9RwQ4klp8ZN8gkuXoFbp8Tf+h8IfpanEA+K+EfEL0=', 'x-amz-request-id': 'RFV6RBG6NDPVD7NY', 'date': 'Thu, 16 Oct 2025 02:26:47 GMT', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'Buckets': [{'Name': 'project1-mlflow-bucket', 'CreationDate': datetime.datetime(2025, 10, 12, 12, 4, 13, tzinfo=tzlocal())}], 'Owner': {'ID': '866466d6c4d8a9893e39cdce3c468d75133f23de2c907deeda4f1a78a7ca565b'}}


In [4]:
import mlflow
mlflow.set_tracking_uri("http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/")

print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/


In [5]:
mlflow.set_experiment("exp 4 - Handling imbalanced dataset")

2025/10/16 02:29:49 INFO mlflow.tracking.fluent: Experiment with name 'exp 4 - Handling imbalanced dataset' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://project1-mlflow-bucket/443594013549356635', creation_time=1760581789462, experiment_id='443594013549356635', last_update_time=1760581789462, lifecycle_stage='active', name='exp 4 - Handling imbalanced dataset', tags={}>

In [6]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN            # ENN = Extended Nearest Neighbor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [7]:
url = 'https://raw.githubusercontent.com/adityasahusomu/Youtube_Comment_Analyzer/refs/heads/main/cleaned_reddit_dataset.csv'
df = pd.read_csv(url).dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [12]:
def run_imbalanced_experiment(imbalance_method):
    ngram_range = (1, 3)  # Trigram
    max_features = 10000  # max_features = 1000 (TF-IDF)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)

    if imbalance_method == 'class_weights':
        class_weight = 'balanced'
    else:
        class_weight = None


        if imbalance_method == 'oversampling':
            smote = SMOTE(random_state=42)
            X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'adasyn':
            adasyn = ADASYN(random_state=42)
            X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'undersampling':
            rus = RandomUnderSampler(random_state=42)
            X_train_vec, y_train = rus.fit_resample(X_train_vec, y_train)
        elif imbalance_method == 'smote_enn':
            smote_enn = SMOTEENN(random_state=42)
            X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

    with mlflow.start_run() as run:
        mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "imbalance_handling")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("imbalance_method", imbalance_method)

        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
        model.fit(X_train_vec, y_train)

        # Next step= logging metrics
        y_pred = model.predict(X_test_vec)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
        confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
        plt.savefig(confusion_matrix_filename)
        mlflow.log_artifact(confusion_matrix_filename)
        plt.close()

        # model logging
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

# Running experiments for different imbalance methods
imbalance_methods = ['class_weights', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']

for method in imbalance_methods:
    run_imbalanced_experiment(method)


2025/10/16 03:13:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 03:13:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weights_RandomForest_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635/runs/41d9022621a34e479adc918a3b36961d
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635


2025/10/16 03:14:23 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 03:14:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635/runs/e48f9c0c27b04e94b3f017d63c7f7c43
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635


2025/10/16 03:15:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 03:15:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635/runs/e02e7a199230493198ecbde9b71ec9b7
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635


2025/10/16 03:15:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 03:15:53 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635/runs/1b986bc59da94206a85f5d52b5270f53
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635


2025/10/16 03:16:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/10/16 03:17:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635/runs/ffabe1cd34164c02b213eaf4ddb7e1a4
🧪 View experiment at: http://ec2-3-15-32-230.us-east-2.compute.amazonaws.com:5000/#/experiments/443594013549356635
